PokeAPI data analysis on pokemon types and prefered attributes on early generations

at first we will set all the libraries and variables used along

In [45]:
import requests, json, pandas as pd, plotly.express as px

In [46]:
#base url for usage in functions
base_url = 'https://pokeapi.co/api/v2/'
#library of pokemon info
pokemon_info = {'Name': [], 'Types': [], 'Attribute': [], 'IV': [],}

Then, we will do functions to filter the pokemons we wish to use, and functions to gather the info about types and its IV's

In [47]:
#function to gather pokemon per gen(made to also get other gens)
def catch_pokemon_gen(n):
    url = f'https://pokeapi.co/api/v2/generation/{n}'
    p = requests.get(url)
    info = p.json()
    poke_list = []

    for pokemon in info['pokemon_species']:
        name = pokemon['name']
        poke_list.append(name)

    return poke_list

gen1 = catch_pokemon_gen(1)
gen2 = catch_pokemon_gen(2)
gen3 = catch_pokemon_gen(3)
gen4 = catch_pokemon_gen(4)

In [48]:
#function to gather pokemon types
def catch_pokemon_type(name):
    url = base_url + 'pokemon/' + name
    p = requests.get(url)
    info = p.json()
    types = [type['type']['name'] for type in info['types']]#since a pokemon can have 2 types, we read both and add them in a list
    return types

In [49]:
#function to gather pokemon info on its attributes
def catch_pokemon_iv(name):
    url = base_url + 'pokemon/' + name
    p = requests.get(url)
    info = p.json()
    attributes = ['hp', 'attack', 'defense', 'special-attack', 'special-defense', 'speed']
    highest_attribute = attributes[0]
    highest_attribute_iv = 0
    
    for i in range(len(attributes)):
        iv_value = info['stats'][i]['base_stat']

        if iv_value > highest_attribute_iv:
            highest_attribute_iv = iv_value
            highest_attribute = attributes[i]

    return highest_attribute, highest_attribute_iv

Now, using a function we shall aggregate all info we gather into the dictionary

In [50]:
#function for gathering info in a DF
def catch_all_info(list):
    for pokemon in list:
        try:
            types = catch_pokemon_type(pokemon)
            attribute, iv = catch_pokemon_iv(pokemon)
            pokemon_info['Name'].append(pokemon)
            pokemon_info['Types'].append(types)
            pokemon_info['Attribute'].append(attribute)
            pokemon_info['IV'].append(iv)
        except Exception as e:
            print(f'error on pokemon {pokemon}')
    return pokemon_info

In [51]:
gen1_info = catch_all_info(gen1)
gen1df = pd.DataFrame(gen1_info)
gen1df.index += 1
display(gen1df)
pokemon_info = {'Name': [], 'Types': [], 'Attribute': [], 'IV': [],}

,Name,Types,Attribute,IV
1,bulbasaur,"[grass, poison]",special-attack,65
2,charmander,[fire],speed,65
3,squirtle,[water],defense,65
4,caterpie,[bug],hp,45
5,weedle,"[bug, poison]",speed,50
...,...,...,...,...
147,dragonair,[dragon],attack,84
148,dragonite,"[dragon, flying]",attack,134
149,mew,[psychic],hp,100
150,ivysaur,"[grass, poison]",special-attack,80


In [52]:
gen2_info = catch_all_info(gen2)
gen2df = pd.DataFrame(gen2_info)
gen2df.index += 1
display(gen2df)
pokemon_info = {'Name': [], 'Types': [], 'Attribute': [], 'IV': [],}

,Name,Types,Attribute,IV
1,chikorita,[grass],defense,65
2,cyndaquil,[fire],speed,65
3,totodile,[water],attack,65
4,sentret,[normal],attack,46
5,hoothoot,"[normal, flying]",hp,60
...,...,...,...,...
96,blissey,[normal],hp,255
97,pupitar,"[rock, ground]",attack,84
98,tyranitar,"[rock, dark]",attack,134
99,bayleef,[grass],defense,80


In [53]:
gen3_info = catch_all_info(gen3)
gen3df = pd.DataFrame(gen3_info)
gen3df.index += 1
display(gen3df)
pokemon_info = {'Name': [], 'Types': [], 'Attribute': [], 'IV': [],}

error on pokemon deoxys


,Name,Types,Attribute,IV
1,treecko,[grass],speed,70
2,torchic,[fire],special-attack,70
3,mudkip,[water],attack,70
4,poochyena,[dark],attack,55
5,zigzagoon,[normal],speed,60
...,...,...,...,...
130,groudon,[ground],attack,150
131,rayquaza,"[dragon, flying]",attack,150
132,jirachi,"[steel, psychic]",hp,100
133,delcatty,[normal],speed,90


In [54]:
gen4_info = catch_all_info(gen4)
gen4df = pd.DataFrame(gen4_info)
gen4df.index += 1
display(gen4df)
pokemon_info = {'Name': [], 'Types': [], 'Attribute': [], 'IV': [],}

error on pokemon wormadam
error on pokemon giratina
error on pokemon shaymin


,Name,Types,Attribute,IV
1,turtwig,[grass],attack,68
2,chimchar,[fire],speed,61
3,piplup,[water],special-attack,61
4,starly,"[normal, flying]",speed,60
5,bidoof,[normal],hp,59
...,...,...,...,...
100,froslass,"[ice, ghost]",speed,110
101,monferno,"[fire, fighting]",speed,81
102,gastrodon,"[water, ground]",hp,111
103,toxicroak,"[poison, fighting]",attack,106


Here, we create dataframes for each generation we wish to use for our analysis

We then combine all pokemons into a single dataframe for a overall view

In [55]:
#sum of generations into a single DataFrame
total = [gen1df, gen2df, gen3df, gen4df]
totaldf = pd.concat(total)
totaldf = totaldf.reset_index()
display(totaldf)

,index,Name,Types,Attribute,IV
0,1,bulbasaur,"[grass, poison]",special-attack,65
1,2,charmander,[fire],speed,65
2,3,squirtle,[water],defense,65
3,4,caterpie,[bug],hp,45
4,5,weedle,"[bug, poison]",speed,50
...,...,...,...,...,...
484,100,froslass,"[ice, ghost]",speed,110
485,101,monferno,"[fire, fighting]",speed,81
486,102,gastrodon,"[water, ground]",hp,111
487,103,toxicroak,"[poison, fighting]",attack,106


Then, by isolating the types and its values, we can see the gamefreak devs preferences towards some pokemon types

In [56]:
#overall type DataFrame
total_types = totaldf['Types'].str[0]
total_types = total_types.value_counts()
types_df = pd.DataFrame(total_types).reset_index()
types_df = types_df.astype({'Types':'str', 'count': 'int'})

In [57]:
#overall types plot
type_colors = { 'water':'#6890F0','normal':'#A8A878','bug':'#A8B820','grass':'#78C850','fire':'#F08030','electric':'#F8D030','psychic':'#F85888','poison':'#A040A0','rock':'#B8A038','ground':'#E0C068','fighting':'#C03028','fairy':'#EE99AC','ice':'#98D8D8','dark':'#705848','ghost':'#705898','dragon':'#7038F8','steel':'#B8B8D0'}
type_plot = px.pie(types_df, names='Types', values='count',color='Types',color_discrete_map=type_colors)
type_plot.show()

During the early generations of pokemon, there was a preference of the devs to opt for water pokemons and normal ones

Doing the same with the pokemon attributes, we measure the strongest attribute in every pokemon

In [58]:
#overall attributes DataFrame
total_attributes = totaldf['Attribute']
total_attributes = total_attributes.value_counts()
attributes_df = pd.DataFrame(total_attributes).reset_index()

In [59]:
#overall attributes plot
attribute_colors= {'attack':'#DC143C','special-attack':'#6495ED','speed':'#E9967A','defense':'#FFD700','hp':'#B22222','special-defense':'#8FBC8F'}
attribute_plot = px.bar(attributes_df, x = 'Attribute', y = 'count', color='Attribute',color_discrete_map = attribute_colors)
attribute_plot.show()

Looking at the graphs we can see that pokemon in the early generations, used to have more physical attacks than special ones